In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI管道：使用Google Cloud Pipeline组件进行模型训练、上传和部署

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/google_cloud_pipeline_components_model_train_upload_deploy.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/google_cloud_pipeline_components_model_train_upload_deploy.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/pipelines/google_cloud_pipeline_components_model_train_upload_deploy.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

概述

本笔记本展示如何使用[`google_cloud_pipeline_components`](https://github.com/kubeflow/pipelines/tree/master/components/google-cloud)中定义的组件来构建一个[Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines)工作流，该工作流用于训练一个[自定义模型](https://cloud.google.com/vertex-ai/docs/training/containers-overview)，将模型上传为`Model`资源，创建一个`Endpoint`资源，并将`Model`资源部署到`Endpoint`资源上。

了解更多关于[Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)和[自定义训练组件](https://cloud.google.com/vertex-ai/docs/pipelines/customjob-component)的信息。

### 目标

在本教程中，您将学习如何使用`Vertex AI Pipelines`和`Google Cloud Pipeline Components`构建和部署自定义模型。

此教程使用以下 Google Cloud ML 服务：

- `Vertex AI Pipelines`
- `Google Cloud Pipeline Components`
- `Vertex AI Training`
- `Vertex AI Model` 资源
- `Vertex AI Endpoint` 资源

执行的步骤包括：

- 创建一个 KFP 管道：
    - 训练一个自定义模型。
    - 将训练好的模型上传为`Model`资源。
    - 创建一个`Endpoint`资源。
    - 将`Model`资源部署到`Endpoint`资源。
- 编译 KFP 管道。
- 使用`Vertex AI Pipelines`执行 KFP 管道。

这些组件在[此处有文档](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.39/google_cloud_pipeline_components.aiplatform.html)。
(从该页面，还可以查看`CustomPythonPackageTrainingJobRunOp`和`CustomContainerTrainingJobRunOp`组件，这些组件类似地运行'custom'训练，但与相关的`google.cloud.aiplatform.CustomContainerTrainingJob`和`google.cloud.aiplatform.CustomPythonPackageTrainingJob`方法一样，这些方法来自[Vertex AI SDK](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)，也会上传训练好的模型)。

数据集

本教程使用的数据集是[Cloud公共数据集计划](https://cloud.google.com/bigquery/public-data/) [伦敦自行车出租](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=london_bicycles&page=dataset&_ga=2.122237643.-1779725180.1624895157)，结合了[NOAA天气数据](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=noaa_gsod&page=dataset&_ga=2.179861860.-1779725180.1624895157)

该数据集预测自行车出租的持续时间。

### 成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* 云存储

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [云存储 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 基于您的预期使用量生成成本估算。

安装

安装执行此笔记本所需的软件包。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage \
                                 'kfp<2' \
                                 'google-cloud-pipeline-components<2'

只有 Colab：取消注释以下单元格以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

##开始之前

###设置您的谷歌云项目

**无论您使用的是哪种笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建一个帐户时，您将获得$300的免费信用额度，用于支付计算/存储成本。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex AI API]。

4. 如果您正在本地运行此笔记本，您需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下方法：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 参考支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

UUID

如果您正在进行实时教程会话，您可能正在使用共享的测试账户或项目。为了避免用户之间在创建资源时发生名称冲突，您为每个实例会话创建一个uuid，并将其附加到您在本教程中创建的资源的名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的 Google Cloud 账号

根据您的 Jupyter 环境，您可能需要手动进行认证。请按照以下相关说明操作。

**1. Vertex AI Workbench**
* 无需任何操作，因为您已经认证。

**2. 本地 JupyterLab 实例，取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples查看如何向您的服务帐户授予Cloud Storage权限。

创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

- *{给笔记本作者的备注：对于任何需要唯一的用户提供的字符串（如存储桶名称或模型ID），请在末尾添加“-unique”以便进行适当的测试}*

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶尚不存在时才能运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

服务账户

如果您不知道您的服务账户，请尝试使用`gcloud`命令在下面执行第二个单元格来获取您的服务账户。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import os
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

print("Service Account:", SERVICE_ACCOUNT)

### 为Vertex AI Pipelines设置服务账号访问权限

运行以下命令，为您的服务账号授予读取和写入管道工件的访问权限。这个桶是在之前的步骤中创建的，您只需要针对每个服务账号运行一次这些命令。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
from typing import Any, Dict, List

import google.cloud.aiplatform as aip
import kfp
from kfp.v2 import compiler  # noqa: F811

设置以下常量用于Vertex AI管道：

In [ ]:
PIPELINE_ROOT = f"{BUCKET_URI}/pipeline_root/bikes_weather"

初始化Python的Vertex AI SDK

为您的项目和相应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

## 定义自定义模型流水线，使用来自`google_cloud_pipeline_components`的组件

接下来，您定义流水线。

`experimental.run_as_aiplatform_custom_job`方法以先前定义的组件和自定义训练作业配置的`worker_pool_specs`列表作为参数— 在这种情况下是一个。

然后，使用[`google_cloud_pipeline_components`](https://github.com/kubeflow/pipelines/tree/master/components/google-cloud) 组件来定义流水线的其余部分：上传模型，创建端点，并将模型部署到端点。

*注意：*尽管在此示例中未显示，但如果未提供端点，则模型部署将创建一个端点。

In [ ]:
hp_dict: str = '{"num_hidden_layers": 3, "hidden_size": 32, "learning_rate": 0.01, "epochs": 1, "steps_per_epoch": -1}'
data_dir: str = "gs://cloud-samples-data/vertex-ai/pipeline-deployment/datasets/bikes_weather/"
TRAINER_ARGS = ["--data-dir", data_dir, "--hptune-dict", hp_dict]

# create working dir to pass to job spec
WORKING_DIR = f"{PIPELINE_ROOT}/{UUID}"

MODEL_DISPLAY_NAME = f"train_deploy{UUID}"
print(TRAINER_ARGS, WORKING_DIR, MODEL_DISPLAY_NAME)


@kfp.dsl.pipeline(name="train-endpoint-deploy" + UUID)
def pipeline(
    project: str = PROJECT_ID,
    model_display_name: str = MODEL_DISPLAY_NAME,
    serving_container_image_uri: str = "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-9:latest",
):
    from google_cloud_pipeline_components.types import artifact_types
    from google_cloud_pipeline_components.v1.custom_job import \
        CustomTrainingJobOp
    from google_cloud_pipeline_components.v1.endpoint import (EndpointCreateOp,
                                                              ModelDeployOp)
    from google_cloud_pipeline_components.v1.model import ModelUploadOp
    from kfp.v2.components import importer_node

    custom_job_task = CustomTrainingJobOp(
        project=project,
        display_name="model-training",
        worker_pool_specs=[
            {
                "containerSpec": {
                    "args": TRAINER_ARGS,
                    "env": [{"name": "AIP_MODEL_DIR", "value": WORKING_DIR}],
                    "imageUri": "gcr.io/google-samples/bw-cc-train:latest",
                },
                "replicaCount": "1",
                "machineSpec": {
                    "machineType": "n1-standard-16",
                    "accelerator_type": aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
                    "accelerator_count": 2,
                },
            }
        ],
    )

    import_unmanaged_model_task = importer_node.importer(
        artifact_uri=WORKING_DIR,
        artifact_class=artifact_types.UnmanagedContainerModel,
        metadata={
            "containerSpec": {
                "imageUri": "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-9:latest",
            },
        },
    ).after(custom_job_task)

    model_upload_op = ModelUploadOp(
        project=project,
        display_name=model_display_name,
        unmanaged_container_model=import_unmanaged_model_task.outputs["artifact"],
    )
    model_upload_op.after(import_unmanaged_model_task)

    endpoint_create_op = EndpointCreateOp(
        project=project,
        display_name="pipelines-created-endpoint",
    )

    ModelDeployOp(
        endpoint=endpoint_create_op.outputs["endpoint"],
        model=model_upload_op.outputs["model"],
        deployed_model_display_name=model_display_name,
        dedicated_resources_machine_type="n1-standard-16",
        dedicated_resources_min_replica_count=1,
        dedicated_resources_max_replica_count=1,
    )

编译管道。

In [ ]:
compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path="tabular_regression_pipeline.json",
)

运行管道

接下来，运行管道。

In [ ]:
DISPLAY_NAME = "bikes_weather_" + UUID

job = aip.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="tabular_regression_pipeline.json",
    pipeline_root=PIPELINE_ROOT,
    enable_caching=False,
)

job.run(service_account=SERVICE_ACCOUNT)

! rm tabular_regression_pipeline.json

点击生成的链接查看您在云控制台中的运行情况。

<!--当它正在运行时，它应该看起来像这样：

<a href="https://storage.googleapis.com/amy-jo/images/mp/automl_tabular_classif.png" target="_blank"><img src="https://storage.googleapis.com/amy-jo/images/mp/automl_tabular_classif.png" width="40%"/></a> -->

在用户界面中，当您点击它们时，许多管道DAG节点将会展开或折叠。 这是DAG的部分展开视图（点击图像查看更大版本）。

<a href="https://storage.googleapis.com/amy-jo/images/mp/train_endpoint_deploy.png" target="_blank"><img src="https://storage.googleapis.com/amy-jo/images/mp/train_endpoint_deploy.png" width="75%"/></a>

# 清理工作

要清理此项目中使用的所有谷歌云资源，您可以删除用于教程的[谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源 -- *注意:* 这是自动生成的，并非所有资源都适用于此教程：
### 获取清理管道中的资源
用于获取任务详细信息的函数

In [ ]:
def get_task_detail(
    task_details: List[Dict[str, Any]], task_name: str
) -> List[Dict[str, Any]]:
    for task_detail in task_details:
        if task_detail.task_name == task_name:
            return task_detail

In [ ]:
pipeline_task_details = (
    job.gca_resource.job_detail.task_details
)  # fetch pipeline task details


# fetch endpoint from pipeline and delete the endpoint
endpoint_task = get_task_detail(pipeline_task_details, "endpoint-create")
endpoint_resourceName = (
    endpoint_task.outputs["endpoint"].artifacts[0].metadata["resourceName"]
)
endpoint = aip.Endpoint(endpoint_resourceName)
# undeploy model from endpoint
endpoint.undeploy_all()
endpoint.delete()

# fetch model from pipeline and delete the model
model_task = get_task_detail(pipeline_task_details, "model-upload")
model_resourceName = model_task.outputs["model"].artifacts[0].metadata["resourceName"]
model = aip.Model(model_resourceName)
model.delete()

job.delete()

In [ ]:
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI